# Model Registration Example

This notebook will be a demonstration/example of how one would train a model and log it to the MLflow tracking server.

In [1]:
import pandas as pd
import numpy as np

import ames_notebooks
from app.data_ingestion.read_data import DataReader
from app.pipelines.training import evaluate_model
from app.config.settings import settings

2025-11-26 18:12:11.085 | DEBUG    | app.config.settings:<module>:36 - loaded settings: {
    "DATA_DIRECTORY": "data",
    "RAW_DATA_DIRECTORY": "data/raw",
    "PROCESSED_DATA_DIRECTORY": "data/processed",
    "KAGGLE_COMPETITION": "house-prices-advanced-regression-techniques",
    "KAGGLE_DOWNLOAD_PATH": "data/house-prices-advanced-regression-techniques.zip",
    "PROD_MODEL_NAME": "prod",
    "LOG_LEVEL": "INFO",
    "LOG_FILE": "logs/app.log",
    "CATEGORICAL_NAN": "N/A",
    "MLFLOW_EXPERIMENT_NAME": "ames-housing-pricing-experiment",
    "MLFLOW_TRACKING_URI": "https://ames-housing-mlflow-production.up.railway.app"
}


In [2]:
reader = DataReader()
train, test = reader.load_train_test()
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,N/A,Reg,Lvl,AllPub,Inside,...,0,N/A,N/A,N/A,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,N/A,Reg,Lvl,AllPub,FR2,...,0,N/A,N/A,N/A,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,N/A,IR1,Lvl,AllPub,Inside,...,0,N/A,N/A,N/A,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,N/A,IR1,Lvl,AllPub,Corner,...,0,N/A,N/A,N/A,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,N/A,IR1,Lvl,AllPub,FR2,...,0,N/A,N/A,N/A,0,12,2008,WD,Normal,250000


### Create classes for feature engineering, transformations, etc

If we use we want to "package" our pipeline so that we can hand it off to MLflow (so that we don't have hard-coded data logic living on the API), we must define custom classes in a separate .py file (outside of a notebook).

[Learn more](https://mlflow.org/docs/latest/ml/model/dependencies/)

In [3]:
from notebooks.example_utils.custom_transformers import MyFeatureEngineer

### Create a preprocessing pipeline

This should contain all the logic for what transformations need to be made on raw input data *before* inference.

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

feature_engineer = MyFeatureEngineer()

# Create preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=settings.CATEGORICAL_NAN, strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

column_transformer = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, make_column_selector(dtype_include=['int64', 'float64'])),
        ('cat', categorical_transformer, make_column_selector(dtype_include=['string']))
        ]
    )

# build pipeline
preprocessing_pipeline = Pipeline([
        ('feature_engineer', MyFeatureEngineer()),
        ('column_transformer', column_transformer),
        ]
    )

### Build your model

In [5]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# check how well model performs, cross validate, etc...

from sklearn.model_selection import train_test_split
X, y = train.drop('SalePrice', axis=1), train['SalePrice']
X_processed = preprocessing_pipeline.fit_transform(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train).score(X_val, y_val)

-0.0650247100702408

### Create the full pipeline

In [6]:
from sklearn.compose import TransformedTargetRegressor

# this is for adding transformations to the target variable (e.g., log)
target_pipeline = TransformedTargetRegressor(
    regressor=model,
    func=np.log1p,          # before inference transform y -> log(1 + y)
    inverse_func=np.expm1   # after inference transform log(1 + y) -> exp(log(1 + y)) - 1 = y
)

full_pipeline = Pipeline(preprocessing_pipeline.steps + [('model', target_pipeline)])
full_pipeline

,steps,"[('feature_engineer', ...), ('column_transformer', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Build and log to MLflow

In [7]:
import mlflow
from datetime import datetime

# only need this if you are running MLflow locally
mlflow.set_tracking_uri("http://127.0.0.1:8500")

mlflow.set_experiment("example-experiment-" + datetime.now().strftime('%d%m%y%H%M%S'))

2025/11/26 18:12:25 INFO mlflow.tracking.fluent: Experiment with name 'example-experiment-261125181220' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts/mlruns/19', creation_time=1764198745237, experiment_id='19', last_update_time=1764198745237, lifecycle_stage='active', name='example-experiment-261125181220', tags={}>

In [8]:
X_train, X_test, y_train, prediction = train_test_split(X, y, test_size=0.2, random_state=42)

with mlflow.start_run(run_name = "example-run"):
    full_pipeline.fit(X_train, y_train)
    
    train_metrics = evaluate_model(full_pipeline, X_train, y_train, prefix="train_")
    test_metrics = evaluate_model(full_pipeline, X_test, prediction, prefix="test_")
    
    # log to mlflow
    model_name = "example-linear-regression"
    mlflow.log_params(model.get_params())
    mlflow.log_metrics({**train_metrics, **test_metrics})
    model_info = mlflow.sklearn.log_model(
        full_pipeline,
        name=model_name,
        registered_model_name=model_name,
        input_example=X_train,
        code_paths=["example_utils"] # important!
    )

/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [14, 29] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-valu

🏃 View run example-run at: http://127.0.0.1:8500/#/experiments/19/runs/577f268850274212a8fbd62056e88a2b
🧪 View experiment at: http://127.0.0.1:8500/#/experiments/19


In [9]:
from app.inference.predict import get_model

logged_model = get_model('prod')

/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1124: UserWarning: [18:12:43] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  self.get_booster().load_model(fname)
2025-11-26 18:12:43.451 | INFO     | app.inference.predict:get_model:24 - loaded model with id m-0469ab7ba0ea4bb2a471785a25ed796c


In [12]:
logged_model.model_id

'm-0469ab7ba0ea4bb2a471785a25ed796c'

In [13]:
search = mlflow.search_registered_models(filter_string=f"name = 'prod'")
[x.latest_versions[0].version for x in search]


['6']

In [24]:

search = mlflow.search_registered_models(filter_string=f"name = 'prod'")
result = search[0]
result.latest_versions[-1].source

'models:/xgboost-baseline/5'

In [58]:
logged_model.metadata.get_model_info().name

'xgboost-optimized'

In [ ]:
logged_model.predict(X)

/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [14, 29] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array([205029.05755346, 211290.19423459, 218422.43152712, ...,
       266861.50266879, 139638.84449723, 144322.51789733], shape=(1460,))

### Tag model with Kaggle submission score

In [ ]:
from app.pipelines.training import submit_to_kaggle_and_tag

prediction = logged_model.predict(test)

submission = pd.DataFrame({
    'Id': test.index,
    'SalePrice': prediction
})

submit_to_kaggle_and_tag(submission, model_name)

/Users/nic/git/AmesHousingPredictor/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 5, 15, 16, 30, 31, 41] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
2025-11-26 15:54:48.210 | DEBUG    | app.pipelines.kaggle_utils:submit_to_kaggle:38 - Kaggle submission file saved to ../submissions/submission_11-26-25_15:54:48.csv
2025-11-26 15:54:48.211 | DEBUG    | app.pipelines.kaggle_utils:submit_to_kaggle:42 - Submitting submission 11-26-25_15:54:48 to Kaggle
100%|██████████| 33.6k/33.6k [00:00<00:00, 87.5kB/s]


('example-linear-regression', '4', '0.17906')